In [10]:
import openai

PROMPT = """
I have the following functions in my system.

'get_weather'
'get_currency'
'get_news'

All of them receive the name of a country as an argument (i.e. get_weather('Spain'))

Please, answer with the name of the function that you would like me to run.

Please, say nothing else, just the name of the function with the arguments.

Answer the following question:

What is the weather in Korea?
"""

client = openai.OpenAI()
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": PROMPT
        }
    ]
)


response.choices[0].message.content

"get_weather('Korea')"